In [5]:
# SpaceRanger:

In [6]:
# ./run_spaceranger.single.sh p96_p1_run3 P96_P1 P96_P1 Visium.image/jpg/P96P_updated.jpeg V12D14-351 D1 /genomics/1_Projects/Ecig_U01/Visium/visium_final_count/96P_json/V12D14-351-D1.json

# using 

#./spaceranger-2.1.1/spaceranger count --id  <Folder path to store results> --transcriptome <Folder path containing Rat transcriptome information and reference in JSON format> --fastqs <Folder path containing fastq files> --sample <sample filename prefix> --include-introns true --localmem 300 --localcores 60 --image <File path of the slice/sample figure in JPG or TIFF format> --slide <slide code name> --area <area code> --loupe-alignment <File path of the Loupe alignment in JSON format>

In [ ]:
# Velocyto:

In [7]:
# velocyto run10x -@ 60 --samtools-memory 100000 <Folder path to SpaceRanger results> <File path of the Rat genes gtf>


In [ ]:
# Data cleaning and Integration 

In [ ]:
import scvi
import os
from scipy.sparse import csr_matrix
import diffxpy.api as de

PATH_TO_INPUT_DATA="/genomics/Rachid/Velocity/"
FILE_VELO_ADATA="velo_mp_integrated.July16.h5ad"
MODEL_VELO_ADATA="model_integrated_velo_mp.July16.model"

d_sample_to_dirname = {
        "P94P1":"p94_p1_run2",
        "P95P1":"p95_p1_run2",
        "P100P1":"p100_p1_run2",
        "P81P2":"p81_p2_run2",
        "P81P1":"p81_p1_run2",
        "P83P1":"p83_p1_run2",
        "P112P1":"p112_p1_run2",
        "P113P1":"p113_p1_run2",
        "P80P1":"p80_p1_run2",
        }

d_sample_to_categ = {
        "P94P1":"CM",
        "P95P1":"CM",
        "P100P1":"CM",
        "P81P2":"EcigM",
        "P81P1":"EcigM",
        "P83P1":"EcigM",
        "P112P1":"EcigM",
        "P113P1":"EcigM",
        "P80P1":"EcigM",
        }

def preprocess_data(folder_name, loom_file, idx_sample):
    adata = sc.read_10x_mtx(folder_name)
    sc.pp.filter_cells(adata, min_genes=200)
    adata.var['mt'] = adata.var_names.str.startswith('Mt-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
    lower_lim = np.quantile(adata.obs.n_genes_by_counts.values, .02)
    upper_lim_pct = np.quantile(adata.obs.pct_counts_mt.values, .98)
    lower_lim_pct = np.quantile(adata.obs.pct_counts_mt.values, .02)
    print(f"genes_by_counts: {lower_lim} to {upper_lim}, pct_mt: {lower_lim_pct} to {upper_lim_pct}")
    print(folder_name + ": Original size: " + str(len(adata)))
    adata = adata[(adata.obs.n_genes_by_counts < upper_lim) & (adata.obs.n_genes_by_counts > lower_lim)]
    print(folder_name + ": After filtering per n_genes_by_counts: " + str(len(adata)))
    adata = adata[adata.obs.pct_counts_mt < upper_lim_pct, :]
    print(folder_name + ": After filtering per pct_counts_mt: " + str(len(adata)))

    ldata = scv.read(loom_file)
    adata = scv.utils.merge(adata, ldata)
    scv.pp.filter_and_normalize(adata)
    scv.pp.moments(adata)
    scv.tl.velocity(adata)
    adata.obs.pop("initial_size_unspliced")
    adata.obs.pop("initial_size_spliced")
    adata.obs.pop("initial_size")
    adata.obs["Sample"] = idx_sample
    return adata

def run_integration():
    if not os.path.exists(FILE_VELO_ADATA):
        adata_AF = []
        for (s_name, s_dirname) in d_sample_to_dirname.items():
            print("Adding " + s_name)
            adata_AF.append(
                    preprocess_data(
                        PATH_TO_INPUT_DATA + s_dirname + "/outs/filtered_feature_bc_matrix/",
                        PATH_TO_INPUT_DATA + s_dirname + "/velocyto/" + s_dirname +".loom" ,
                        s_name
                        )
                    )
        adata = sc.concat(adata_AF)
        adata.obs["Condition"] = adata.obs.Sample.map(d_sample_to_categ)

        sc.pp.filter_genes(adata, min_cells=200)

        layers = ["spliced", "unspliced", "ambigious"]
        layers_keys = [key for key in layers if key in adata.layers.keys()]
        counts_layers = [np.sum(adata.layers[key], axis=1) for key in layers_keys]
        counts_total = np.sum(counts_layers, 0)
        counts_total += counts_total == 0
        counts_layers = np.array([counts / counts_total for counts in counts_layers])
        adata.obs["U_US_ratio"] = counts_layers[1]

        adata.X = csr_matrix(adata.X)
        adata.obs.groupby("Sample").count()
        # Saving Raw data
        adata.layers["counts"] = adata.X.copy()
        adata.raw_unnormalized = adata.copy()

        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata)

        adata.raw = adata.copy()

        print("Running Integration of samples")
        # Integration method:
        scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                categorical_covariate_keys = ["Sample"],
                continuous_covariate_keys = ["pct_counts_mt", "n_counts"])
        model = scvi.model.SCVI(adata)
        model.train()
        model.get_latent_representation().shape
        adata.obsm["X_scVI"] = model.get_latent_representation()
        sc.pp.neighbors(adata, use_rep = "X_scVI")

In [ ]:
# Merging RNA velocity data from each sample into one integrated AnnData instance:

In [4]:
import pandas as pd
import scvelo as scv
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

MALEDATATYPE="fp"
FILE_VELO_MA_EXT_ADATA="./velo_" + MALEDATATYPE + "_integrated.July16.h5ad"
FILE_VELO_UPD_M_ADATA_PRE="./VersionJuly1/velo_"+ MALEDATATYPE + "_anndata/"


def merge_anndata_objects(intg_adata_ext, ldata):
    try:
        adata_vz = scv.utils.merge(intg_adata_ext, ldata)
    except AttributeError as e:
        # Custom debug message for AttributeError
        print(f"AttributeError encountered: {e}")
        # Detailed debugging
        for k in intg_adata_ext.obs:
            print(f"Column: {k}, Dtype: {intg_adata_ext.obs[k].dtype}")

        for k in ldata.obs:
            print(f"Column: {k}, Dtype: {ldata.obs[k].dtype}")
        raise
    return adata_vz


def rotate_coordinates(x, y, angle_degrees = 180):
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)

    # Calculate the midpoint of the dataset on the x-axis and y-axis
    midpoint_x = np.mean(x)
    midpoint_y = np.mean(y)

    # Convert the angle from degrees to radians
    angle_radians = np.radians(angle_degrees)

    # Create the rotation matrix
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians)],
        [np.sin(angle_radians), np.cos(angle_radians)]
        ])

    # Center the points around the midpoint
    centered_points = np.vstack((x - midpoint_x, y - midpoint_y))

    # Rotate the points
    rotated_points = rotation_matrix.dot(centered_points)

    # Shift the points back to the original midpoint
    rotated_x = rotated_points[0, :] + midpoint_x
    rotated_y = rotated_points[1, :] + midpoint_y

    # Convert the rotated coordinates back to lists
    return rotated_x.tolist(), rotated_y.tolist()


def generate_ann_data_sample(folder_name, sample_code, fig_sample_name, coord_filename):
    print("START: Generating AnnData for " + fig_sample_name + " ... ")
    intg_adata = sc.read_h5ad(FILE_VELO_MA_EXT_ADATA)
    path_prefix_file = "./" + folder_name + "/" + sample_code + "_run2"

# Read the CSV file containing morphological annotations for each spot:
    csv_file = pd.read_csv("morph_mapping/" + coord_filename)

# Extract the first column (index column)
    index_column = csv_file.iloc[:, 0].astype(str).str[:12]
    intg_adata_proj = intg_adata[intg_adata.obs.Sample == fig_sample_name]
    filtered_adata = intg_adata_proj[intg_adata_proj.obs_names.isin(index_column)]
    filtered_adata_df = filtered_adata.to_df()
    csv_file = csv_file.set_index(csv_file.columns[0])

# Use the first 12 characters of the CSV index to match the filtered AnnData index
    csv_file.index = csv_file.index.astype(str).str[:12]
    result_df = csv_file.join(filtered_adata_df, how='inner')
    result_df_col6 = result_df[["row", "col", "imagerow", "imagecol", "region"]]

# Ensure both have the same index (common key)
    intg_adata_proj.obs_names = intg_adata_proj.obs_names.astype(str)
    result_df_col6.index = result_df_col6.index.astype(str)

# Subset the AnnData object to include only rows that match the metadata DataFrame's index
    intg_adata_ext = intg_adata_proj[intg_adata_proj.obs_names.isin(result_df_col6.index)]

# Add the metadata columns to the AnnData object's obs
    for col in result_df_col6.columns:
        intg_adata_ext.obs[col] = result_df_col6.loc[intg_adata_ext.obs_names, col]

    intg_adata_ext.obs.rename(columns={"region": "Regions"}, inplace=True)
    ldata = scv.read(path_prefix_file + ".loom")

    adata_vz =  merge_anndata_objects(intg_adata_ext, ldata)

    positions = pd.read_csv("./SpaceRangerTissuePositions/" + sample_code + "_run2tissue_positions.csv")
    positions = positions.tail(positions.shape[0] -1)
    positions.index = positions.index.rename("CellID")
    positions["CellID"] = [a[:-6] for a in positions["barcode"]]
    positions.drop("barcode", inplace=True, axis=1)
    positions.index = positions["CellID"]
    positions['pxl_col_in_fullres'] = positions['pxl_col_in_fullres'].astype('float32')
    positions['pxl_row_in_fullres'] = positions['pxl_row_in_fullres'].astype('float32')

    new_x = positions['pxl_row_in_fullres']
    new_y = positions['pxl_col_in_fullres']

    if fig_sample_name in ["P93A1", "P81A2", "P100P1", "P112P1", "P113P1", "P112A1", "P100A1"]:
        print("transforming grid coords...")
        new_x_2, new_y_2 = rotate_coordinates(new_x, new_y)

    positions['pxl_row_in_fullres'] = new_x_2 
    positions['pxl_col_in_fullres'] = new_y_2

    adata_vz.obs = adata_vz.obs.join(positions, how="left")

    adata_vz.obsm['X_umap'] = adata_vz.obs[
            ['pxl_row_in_fullres', 'pxl_col_in_fullres']
            ].to_numpy()

    warnings.filterwarnings('ignore')
    scv.pp.filter_and_normalize(adata_vz)
    scv.pp.moments(adata_vz)

    warnings.filterwarnings('ignore')
    sc.pp.neighbors(adata_vz)
    scv.tl.velocity(adata_vz)
    scv.tl.velocity_graph(adata_vz)
    scv.set_figure_params()
    scv.tl.recover_dynamics(adata_vz, n_jobs = 40)
    scv.tl.velocity(adata_vz, mode="dynamical")
    scv.tl.velocity_graph(adata_vz)
    scv.pl.velocity_embedding_stream(adata_vz, basis = "umap", color = "Regions", save= "_intg_" + fig_sample_name +"_dyn_stream")
    scv.tl.latent_time(adata_vz)
    scv.pl.scatter(adata_vz, color = "latent_time", color_map = "gnuplot", size=80, save=  "_intg_" + fig_sample_name + "_latent_time")
    warnings.filterwarnings('ignore')
    scv.pl.velocity_embedding_grid(adata_vz, basis = "umap", color = "Regions", arrow_length=1, arrow_size=1, dpi=120, save = "_intg_" + fig_sample_name + "_grid_velo")

    warnings.filterwarnings('ignore')
    scv.tl.velocity_pseudotime(adata_vz)
    scv.pl.scatter(adata_vz, color='velocity_pseudotime', cmap='gnuplot', save = "_intg_" + fig_sample_name + "_velo_pseudo_time")

    scv.tl.velocity_confidence(adata_vz)
    keys = "velocity_length", "velocity_confidence"
    scv.pl.scatter(adata_vz, c = keys, perc = [5, 95],  save = "_intg_" + fig_sample_name + "_velo_length_confidence")
    scv.pl.scatter(adata_vz, color="U_US_ratio", save = "_intg_" + fig_sample_name + "_u_us_ratio")

    adata_vz.write_h5ad(FILE_VELO_UPD_M_ADATA_PRE + fig_sample_name + ".h5ad")

    print("END: AnnData for " + fig_sample_name + " created.")

if False:
    generate_ann_data_sample("fa_velocyto_loom", "p101_a1", "P101A1", "FA101.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p108_a1", "P108A1", "FA108.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p111_a1", "P111A1", "FA111.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p74_a1", "P74A1", "FA74.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p77_a1", "P77A1", "FA77.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p82_a1", "P82A1", "FA82.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p93_a1", "P93A1", "FA93.cor.csv")
    generate_ann_data_sample("fa_velocyto_loom", "p96_a1", "P96A1", "FA96.cor.csv")


In [2]:
# Differential Analysis

In [3]:
import seaborn as sns
import math
import scipy
import matplotlib.pyplot as plt

from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import scanpy as sc
import scvelo as scv
import numpy as np
import matplotlib.pyplot as plt
import scvi
import os
from scipy.sparse import csr_matrix
import diffxpy.api as de

#FA
FILE_VELO_FA_COMP_ADATA="velo_fa_integrated.completed.July10.genes_velo.h5ad"
FILE_VELO_FA_ADATA="velo_fa_integrated.completed.July1.genes.h5ad"
MODEL_VELO_FA_COMP_ADATA="model_integrated_velo_fa.completed.July10.genes_velo.model"
#MA
FILE_VELO_MA_COMP_ADATA="velo_ma_integrated.completed.July11.genes.h5ad"
FILE_VELO_MA_ADATA="velo_ma_integrated.completed.July11.h5ad"
MODEL_VELO_MA_COMP_ADATA="model_integrated_velo_ma.completed.July11.genes.model"

#MP
FILE_VELO_MP_COMP_ADATA="velo_mp_integrated.completed.July16.genes_velo.h5ad"
FILE_VELO_MP_ADATA="velo_mp_integrated.completed.July16.h5ad"
MODEL_VELO_MP_COMP_ADATA="model_integrated_velo_mp.completed.July16.genes_velo.model"

#FP
FILE_VELO_FP_COMP_ADATA="velo_fp_integrated.completed.July16.genes_velo.h5ad"
FILE_VELO_FP_ADATA="velo_fp_integrated.completed.July16.h5ad"
MODEL_VELO_FP_COMP_ADATA="model_integrated_velo_fp.completed.July16.genes_velo.model"

def run_integration():
    if not os.path.exists(FILE_VELO_MP_COMP_ADATA):
        adata = sc.read_h5ad(FILE_VELO_MP_ADATA)
        # Removal of cells with no recognized regions
        vals = adata.obs['Regions'].unique()
        res = []
        for v in range(len(vals)):
            if len(vals[v]) > 8:
                res.append(vals[v])
        adata = adata[adata.obs['Regions'].isin(res) == False]
        adata.obs.drop(
            columns=["leiden", "_scvi_batch", "_scvi_labels", "pxl_col_in_fullres", "pxl_row_in_fullres"],
            inplace=True,
        )
        # Remove Velocity-based attribbutes
        #adata.obs.drop(
        #    columns=["latent_time", "velocity_length", "velocity_pseudotime", "U_US_ratio"],
        #    inplace=True,
        #)
        adata.X = csr_matrix(adata.X)
        adata.obs.groupby("Sample").count()
        # Saving Raw data
        adata.layers["counts"] = adata.X.copy()
        adata.raw_unnormalized = adata.copy()

        #sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.normalize_total(adata)
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata)
        print("We keep highly variable genes, size:")
        print(len(adata.obs))
        print(len(adata.var))

        adata.raw = adata.copy()

        print("Running Integration of samples")
        # Integration method:
        scvi.model.SCVI.setup_anndata(
                adata,
                layer = "counts",
                categorical_covariate_keys = ["Sample"],
                continuous_covariate_keys = ["pct_counts_mt", "n_counts", "latent_time", "velocity_length", "velocity_pseudotime", "U_US_ratio"]
                )
        model = scvi.model.SCVI(adata)
        model.train()
        model.get_latent_representation().shape
        adata.obsm["X_scVI"] = model.get_latent_representation()

        adata.write_h5ad(FILE_VELO_MP_COMP_ADATA)
        model.save(MODEL_VELO_MP_COMP_ADATA)
        return adata, model

# adata, model run_integration()

In [ ]:
import pandas as pd
import scvelo as scv
import scanpy as sc
import numpy as np
import seaborn as sns
import math
import scipy
import matplotlib.pyplot as plt
from scipy import stats

from collections import defaultdict
import warnings
from scipy.stats import chisquare
warnings.filterwarnings('ignore')
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

import diffxpy.api as de
import scvi
import os
from scipy.sparse import csr_matrix
from scipy import stats

FILE_VELO_FP_CMP_GV_ADATA = "../FP/velo_fp_integrated.completed.July16.genes_velo.h5ad"
intg_adata_gv = sc.read_h5ad(FILE_VELO_FP_CMP_GV_ADATA)
MODEL_VELO_FP_CMP_GV_ADATA = "../FP/model_integrated_velo_fp.completed.July16.genes_velo.model/"
model = scvi.model.SCVI.load(MODEL_VELO_FP_CMP_GV_ADATA, intg_adata_gv)

sc.pp.normalize_total(intg_adata_gv)
sc.pp.log1p(intg_adata_gv)
sc.pp.highly_variable_genes(intg_adata_gv, n_top_genes=2000)
sc.tl.rank_genes_groups(intg_adata_gv, "Regions", method='wilcoxon')
sc.pl.rank_genes_groups(intg_adata_gv, n_genes=25, sharey=False)

def find_differentially_veloce_genes(l_adata, l_markers_scvi, name, control_code, treatment_code, min_pvalue = 0.05):
    lcl_subset = l_adata[l_adata.obs["Regions"].isin([name])].copy()
    lcl_subset.X = lcl_subset.X.toarray()
    markers_scvi_lcl = l_markers_scvi[l_markers_scvi["group1"] == name]
    tmp = lcl_subset[lcl_subset.obs.Regions == name]
    for g_name in markers_scvi_lcl.index:
        i = np.where(tmp.var_names == g_name)[0][0]
        a = tmp[tmp.obs.Condition == treatment_code].X[:,i]
        b = tmp[tmp.obs.Condition == control_code].X[:,i]
        pval = stats.mannwhitneyu(a, b).pvalue
        if pval < min_pvalue:
            print("region," + name + ",p-value," + str(pval) + "," + g_name + ",proba_de," + str(markers_scvi_lcl.at[g_name, "proba_de"]))

def run_differential_gene_kinetics(lcl_adata, lcl_model, control_code, treatment_code):  
    markers_1 = sc.get.rank_genes_groups_df(lcl_adata, None)
    markers_1 = markers_1[(markers_1.pvals_adj < 0.05) & (markers_1.logfoldchanges > 0.25)]
    markers_scvi_1 = lcl_model.differential_expression(groupby = "Regions")
    markers_scvi_1 = markers_scvi_1[(markers_scvi_1["is_de_fdr_0.05"]) & (markers_scvi_1.lfc_mean > 0.5)]

#    for name in ["cc", "CPu", "NAc", "AIC", "Septal", "Pir", "Sparse", "ACC", "LV", "Layer1", "Layer2/3", "Layer4", "Layer5/6"]:    
    for name in ["HY","Layer2/3","cc","Amy","Layer4","Layer5/6","TH","Sparse","HP","LV","Layer1","CPu","RSP"]:
        find_differentially_veloce_genes(lcl_adata, markers_scvi_1, name, control_code, treatment_code, 0.05)

# For female data:
run_differential_gene_kinetics(intg_adata_gv ,model, "CF", "EcigF")
# run_differential_gene_kinetics(intg_adata_gv ,model, "CM", "EcigM")